In [2]:
def _info(l):
    d = {}
    d['SEQUENCE LENGTH'] = l[0].split(':')[1].strip()
    d['SEED'] = l[1].split(':')[1].strip()
    d['MIN READ LENGTH'] = l[2].split(':')[1].strip()
    d['MAX READ LENGTH'] = l[3].split(':')[1].strip()
    d['MIN OVERLAP'] = l[4].split(':')[1].strip()
    d['MAX OVERLAP'] = l[5].split(':')[1].strip()
    d['MIN COVERAGE'] = l[6].split(':')[1].strip()
    d['BIPHASIC'] = l[7].split(':')[1].strip()
    d['SEQUENCE'] = l[10].split(':')[1].strip()
    d['READS'] = list(read.strip() for read in l[11].split(':')[1].strip().replace("[","").replace("]","").replace("'","").split(","))
    d['EXTENSION'] = list(extension.strip() for extension in l[12].split(':')[1].strip().replace("[","").replace("]","").replace("'","").split(","))
    d['OVERLAPS'] = l[13].split(':')[1].strip()
    return d

def find_all(s,k):
    i = 0
    while k in s[i:]: 
        i += s[i:].find(k)
        yield i
        i += len(k)

def generate_reads(sequence,min_subseq_len,max_subseq_len,min_overlap,max_overlap,min_coverage=None,circularise=False,seed=None):
    import random

    random.seed(seed)
    if circularise: sequence = sequence[-random.randint(min_overlap,max_overlap):] + sequence + sequence[:random.randint(min_overlap,max_overlap)]
    reads = []
    while 1: 
        start = 0
        end = random.randint(min_subseq_len,max_subseq_len)
        reads += [sequence[start:end]]
        while end < len(sequence):
            start = random.randint(end-max_overlap,end-min_overlap)
            if (len(sequence) - start)/max_subseq_len < 2:
                if (len(sequence) - start)/max_subseq_len < 1:
                    end = len(sequence)
                else:
                    a = 0
                    while (len(sequence) - start)/(min_subseq_len+a) > 2: a+=1
                    end = random.randint(start+min_subseq_len+a,start+max_subseq_len) 
            else: end = random.randint(start+min_subseq_len,start+max_subseq_len) 
            reads += [sequence[start:end]]
        if min_coverage is None or len(set(reads))*(sum(len(read) for read in set(reads))/len(set(reads)))/len(sequence) >= min_coverage: return list(set(reads))


In [3]:
from Bio import SeqIO, Seq
for record in SeqIO.parse("data/input/Raphanus sativus_NC_018551.1.fasta",'fasta'):
    sequence = record.seq
reads = generate_reads(sequence,250,250,50,50,min_coverage=None,seed=0)


In [6]:
sequence[-10:]

Seq('GCTGGAGGAA')

In [39]:
seq = []
with open('data/output/slurm.mscluster65.36535.out','r') as f:
    for line in f: seq += [line]
    info = _info(seq)
    # seq = seq[10:]
    # info[file]['correct'] = list(str(s[:-1] in sequence)+' ('+ str(len(s))+')'  for s in seq)
    # seq = []

In [40]:
info

{'SEQUENCE LENGTH': '258426',
 'SEED': '0',
 'MIN READ LENGTH': '250',
 'MAX READ LENGTH': '250',
 'MIN OVERLAP': '50',
 'MAX OVERLAP': '50',
 'MIN COVERAGE': 'N/A',
 'BIPHASIC': 'False',
 'SEQUENCE': '...',
 'READS': ['...'],
 'EXTENSION': ['...'],
 'OVERLAPS': '...'}

In [48]:
info['EXTENSION'][1] in info['READS']

False

In [50]:
all(read in sequence for read in info["READS"]+[info["SEQUENCE"],info['EXTENSION'][1]])

True

In [49]:
sum(len(read) for read in info["READS"]+[info["SEQUENCE"],info['EXTENSION'][1]])

297436

In [44]:
len(sequence)

258426

In [51]:
sequence

Seq('CCCCTTGAGTCTCTAACATCAGAATCCGAACAAAAGACCTACAGGTCGGCCACA...GAA')

In [57]:
info['EXTENSION'][1] in info["READS"]

False

In [73]:
with open('reads.txt','w') as f:
    for read in info["READS"]+[info["SEQUENCE"],info['EXTENSION'][1]]:
        f.write(read+'\n')

In [69]:
with open('reads_.txt','w') as f:
    for read in reads:
        f.write(read.__str__()+'\n')

In [72]:
info['EXTENSION'][2]

'...'

In [105]:
info["READS"][3][len('AACTATAGAAAGTTGTGGTTGGTTGTTGACCAACAAAAGCATGGGAGAAA'):]

'...'

In [68]:
len(set(reads)) == len(reads)

True

In [74]:
info['EXTENSION'][0]


'AGGAGTAGTGAAGAACTATAGAAAGTTGTGGTTGGTTGTTGACCAACAAAAGCATGGGAGAAA'

In [106]:
info['SEQUENCE'] + info["READS"][3][len('AACTATAGAAAGTTGTGGTTGGTTGTTGACCAACAAAAGCATGGGAGAAA'):] in sequence

True

In [86]:
info['EXTENSION'][2] in info['READS'][2][len(info['EXTENSION'][0]):]

True

In [92]:
sequence.count('AGGAGTAGTGAAGAACTATAGAAAGTTGTGGTTGGTTGTTGACCAACAAAAGCATGGGAGAAA')

2

In [95]:
u[0] + u[1][len('AGGAGTAGTGAAGAACTATAGAAAGTTGTGGTTGGTTGTTGACCAACAAAAGCATGGGAGAAA'):] in sequence

False

In [102]:
for i in find_all(sequence,'AGGAGTAGTGAAGAACTATAGAAAGTTGTGGTTGGTTGTTGACCAACAAAAGCATGGGAGAAA'):
    print(sequence[i-75:i+95])

ATTGAAAGATTCGTCGCATTTTGGATTAATTGATTTCAACAAATCCCTCCAGACAGCTTCATTAGGAAAGCCTCTAGGAGTAGTGAAGAACTATAGAAAGTTGTGGTTGGTTGTTGACCAACAAAAGCATGGGAGAAAACCAAGAACAGGGGGCATAGAGATTTCTTCTC
GTCACCCAGTTGAGGCATGATTAGATTAATTGATTTCAACAAATCCCTCCAGACAGCTTCATTAGGAAAGCCTCTAGGAGTAGTGAAGAACTATAGAAAGTTGTGGTTGGTTGTTGACCAACAAAAGCATGGGAGAAAACCAAGAACAGGGGGCATAGAGATCTCTTCTA


In [103]:
sequence.count('ATTGAAAGATTCGTCGCATTTTGGATTAATTGATTT')

1